In [ ]:
pip install -q yahoo_fin

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
new = []
model = "gstock-4b-test"
tickers = [ 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-5a-test"
new = ['DXC', 'FSLR', 'STLD', 'FICO', 'SRE', 'BKR', 'CTAS', 'PEP', 'ADI', 'ROP' ]
tickers = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
model = "cry-5a-test"
tickers = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']

In [13]:
findata.EPOCHS=200
model = "etf-5b-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
model = "mcap-3b-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS', 'DELL',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']

In [ ]:
pipeline.IS_VERBOSE = False
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [12]:
addAllocHL(df, 0.10, 1)
df1 = df #df[df.Gain > 0]
df1.sort_values('Alloc', ascending=False).head(50)
# df1.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index]
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
23,ETSY,62.30,0.96,1375.70,1437.20,63.97,70.440002,0.10,0.74,1278.98,1579.46,70.650002,0.10,0.70,1027.78,1115.35,58.869999,-0.08,0.10,9.200000
77,WOLF,24.46,0.93,893.98,883.88,25.53,29.360001,0.15,0.77,784.26,875.69,31.379999,0.23,0.71,634.79,630.35,22.889999,-0.10,0.15,8.833333
21,ENPH,43.85,0.93,2320.73,2234.59,114.32,145.399994,0.27,0.61,1345.76,1140.22,129.960007,0.14,0.68,1436.34,1200.11,114.430000,0.00,0.14,8.800000
71,TTD,33.61,0.93,885.55,522.66,94.78,82.660004,-0.13,0.78,914.58,529.84,94.760002,-0.00,0.64,611.12,456.42,82.089996,-0.13,-0.13,8.761538
73,U,23.50,0.91,632.39,1123.85,21.67,27.120001,0.25,0.76,726.84,975.93,31.209999,0.44,0.73,443.82,613.53,22.200001,0.02,0.25,8.740000
46,NVCR,13.93,0.94,1087.74,1338.58,21.99,20.049999,-0.09,0.77,1231.88,1231.14,21.770000,-0.01,0.77,981.94,1076.44,18.520000,-0.16,-0.09,8.733333
27,GFS,39.79,0.95,268.17,311.01,53.95,57.000000,0.06,0.68,230.62,196.44,59.169998,0.10,0.61,110.77,150.32,50.619999,-0.06,0.06,8.666667
4,APP,11.06,0.92,376.78,399.80,82.49,94.300003,0.14,0.75,475.07,229.05,91.550003,0.11,0.75,380.98,379.57,71.949997,-0.13,0.11,8.472727
42,MPWR,219.00,0.91,4147.05,3168.92,730.05,629.429993,-0.14,0.69,3471.70,1601.26,740.809998,0.01,0.69,3273.84,2155.10,625.369995,-0.14,-0.14,8.457143
29,GLBE,18.10,0.92,193.08,330.69,29.13,32.049999,0.10,0.76,201.64,364.01,32.889999,0.13,0.69,214.06,363.06,28.020000,-0.04,0.10,8.400000


In [ ]:
findata.EPOCHS=200
model = "vols-4b-test"
pending = ['ASTR', 'SMWB']
next = []
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CLOV', 'CRCT', 'CXM', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
future = []

In [10]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-4a-test"
next = []
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP', 'AYX',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HIMS','HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-3b-test"
tickers = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']

In [ ]:
findata.EPOCHS=200
model = "com-4a-test"
tickers = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

In [15]:
addAllocHL(df, 0.10, 1)
# df1 = df[df.Gain > 0]
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
# df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
3,DAPP,2.88,0.93,90.54,116.64,9.83,10.900000,0.11,0.66,51.45,88.56,11.500000,0.17,0.61,48.96,38.21,8.380000,-0.15,0.11,8.663636
0,ARKF,14.10,0.91,225.08,164.73,28.10,30.600000,0.09,0.74,177.19,169.54,30.240000,0.08,0.64,157.04,177.21,24.910000,-0.11,0.08,7.975000
2,ARKW,38.11,0.92,683.82,680.04,78.19,83.580002,0.07,0.77,391.79,513.23,83.339996,0.07,0.66,546.03,566.59,67.309998,-0.14,0.07,8.057143
10,IPO,24.38,0.96,292.70,250.85,39.66,41.910000,0.06,0.79,293.09,282.89,42.750000,0.08,0.68,274.28,243.57,35.250000,-0.11,0.06,8.933333
17,JETS,15.00,0.94,119.19,106.33,20.99,22.209999,0.06,0.73,102.90,113.11,22.830000,0.09,0.66,99.30,90.60,20.790001,-0.01,0.06,8.400000
26,TDIV,37.38,0.94,204.94,162.83,72.06,68.070000,-0.06,0.72,155.18,79.33,71.279999,-0.01,0.61,176.50,63.94,68.099998,-0.05,-0.05,8.200000
21,ONLN,26.98,0.91,272.55,318.31,41.74,44.009998,0.05,0.61,48.25,170.48,45.639999,0.09,0.73,213.83,356.93,38.020000,-0.09,0.05,7.300000
35,XLE,25.05,0.91,397.82,242.44,94.96,90.379997,-0.05,0.75,399.62,217.91,96.389999,0.02,0.59,266.87,119.19,89.620003,-0.06,-0.05,7.300000
44,XME,20.00,0.95,314.33,217.04,64.00,61.209999,-0.04,0.74,283.46,136.05,64.320000,0.01,0.57,247.85,107.87,59.349998,-0.07,-0.04,8.250000
12,IXN,36.29,0.90,237.36,181.50,75.90,73.199997,-0.04,0.76,218.26,95.40,75.790001,-0.00,0.68,225.52,169.30,72.059998,-0.05,-0.04,6.500000


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [16]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/


sending incremental file list
etf-5b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-5b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-5b-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,648,597 bytes  received 73 bytes  19,297,340.00 bytes/sec
total size is 202,383,216  speedup is 20.98
